<a href="https://colab.research.google.com/github/GUILLENJV/Pipeline/blob/master/PS_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 679.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
pip install scikeras

In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import os
#import scikeras
#from scikeras.wrappers import KerasRegressor
#from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import load_model
import functools
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [ ]:
df_eval = pd.read_csv(os.path.join(os.path.abspath('model_evaluation.csv')))

In [ ]:
df_eval.head()

,name,model,params,rmse_cv,std_cv,rmse_testset,mae_testset,custom_features,custom_target,all_scores_cv,pipe_file_name
0,RandomForest,"RandomForestRegressor(max_depth=7, min_samples...","[{'regressor__model__n_estimators': 600, 'regr...",336.243,89.084,239.447512,99.571828,True,True,[283.38457272 483.46690151 297.44253447 386.44...,v1_pipe_RandomForest_336.243_20210728_172108.p...
1,XGB,"XGBRegressor(base_score=0.5, booster='gbtree',...","[{'regressor__model__n_estimators': 200, 'regr...",323.863,89.577,231.435812,93.762748,True,True,[274.55360499 475.35215938 279.8041076 369.33...,v1_pipe_XGB_323.863_20210728_172108.pickle
2,LGBM,"LGBMRegressor(boosting_type='dart', colsample_...","[{'regressor__model__num_leaves': 2000, 'regre...",333.078,83.941,236.808509,103.325356,True,True,[270.62498224 476.68977738 294.03818723 376.41...,v1_pipe_LGBM_333.078_20210728_172108.pickle
3,Ridge,Ridge(fit_intercept=False),"[{'regressor__model__fit_intercept': False, 'r...",5381.929,7222.012,2081.096163,745.921845,True,True,[ 1545.12105719 19822.66040318 1689.71159172 ...,v1_pipe_Ridge_5381.929_20210728_172108.pickle
4,NeuralNetwork,<tensorflow.python.keras.wrappers.scikit_learn...,"[{'regressor__model__optimizer': 'Adam', 'regr...",1003.230,958.119,895.182241,262.272394,True,True,[ 461.32625296 2893.62143013 353.9416898 48...,v1_pipe_NeuralNetwork_1003.23_20210728_172108....


Ya que hemos serializado nuestros modelos con las funciones personalizadas, tenemos que cargarlas a nuestro trabajo de modelos.

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [ ]:
def create_model(optimizer='adam', dropout=0.2, activation='relu', kernel_initializer='normal'):
    model = Sequential()
    model.add(Dense(units = 15, activation = activation, input_dim = 15, kernel_initializer=kernel_initializer))
    model.add(Dropout(dropout))
    model.add(Dense(units = 11, activation = activation))
    model.add(Dropout(dropout))
    model.add(Dense(units = 1, activation = activation))

    model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    return model

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import TransformedTargetRegressor

class FeatureCreation(BaseEstimator, TransformerMixin):
    def __init__(self):
        # print('FeatureCreation initialized')
        return None

    # For the fit method, we will pass the parameter x. This is our independent variables.
    # This fit method will be called when we fit the pipeline.
    def fit(self, x, y=None):
        # print('Fit FeatureCreation called')
        return self

    # Here, we will perform all of our transformations. For creating features automatic, we could create parameters in the class and pass the column names to them.
    # But in this case, since it's for this dataset specific, we will perform transformations in the column names directly into the fit method.
    # The transform method is called when we fit and when we predict using the Pipeline. And that's make sense, since we need to create our feature when we will train and when we will predict.
    def transform(self, x, y=None):
        # print('Transform FeatureCreation called')
        # creating a copy to avoid changes to the original dataset
        x_ = x.copy()
        # print(f'Before Transformation: {x_.shape}')
        # and now, we create everyone of our features.
        # Area power of two
        x_['area2'] = x_['area'] ** 2
        # The ratio between area and rooms
        x_['area/room'] = x_['area'] / x_['rooms']
        # The ratio between area and bathroom
        x_['area/bathroom'] = x_['area'] / x_['bathroom']
        # the sum of rooms and bathrooms
        x_['rooms+bathroom'] = x_['rooms'] + x_['bathroom']
        # the product between rooms and bathrooms
        x_['rooms*bathroom'] = x_['rooms'] * x_['bathroom']
        # the ratio between rooms and bathrooms
        x_['rooms/bathroom'] = x_['rooms'] / x_['bathroom']
        # the product between hoa and property tax
        x_['hoa*property tax'] = x_['hoa (R$)'] * x_['property tax (R$)']
        # print(f'After Transformation: {x_.shape}')
        return x_

In [ ]:
# df = pd.read_csv()
df = pd.read_csv(os.path.join(os.path.abspath('houses_to_rent_v2_fteng.csv')))
x = df.drop(columns=['rent amount (R$)'], axis=1)
y = df['rent amount (R$)']
# check if the random state it is equal to when it was trained, this is very important.
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=0)

In [ ]:
df.shape

(8995, 11)

In [ ]:
print(x_train.shape,x_test.shape)

(6746, 10) (2249, 10)


In [ ]:
!mkdir models

# Load the models

In [ ]:
import pickle
import os
from functools import reduce
from tensorflow.keras.models import load_model
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
# for file in os.listdir(os.path.join(os.path.abspath(''), "models")):
rep = {"pipe": "model", "pickle": "h5"}
models_list = os.listdir(os.path.abspath('../content/models'))
for index, row in df_eval.iterrows():
    # check if the file_name is in our models directory
    if row['pipe_file_name'] in models_list:
        # now, load the model.
        with open(os.path.join(os.path.abspath('../content/models'), row['pipe_file_name']), 'rb') as fid:
          model_trained = pickle.load(fid)

        # for the keras model, we have to load the model separately and add into the pipeline or transformed target object.
        if row['name'] == 'NeuralNetwork':
            model_keras = load_model(os.path.join(os.path.abspath('../content/models'), functools.reduce(lambda a, kv: a.replace(*kv), rep.items(), row['pipe_file_name'])))
            # check if the target transformer it is active
            if row['custom_target']:
                # reconstruct the model inside a kerasregressor and add inside the transformed target object
                model_trained.regressor.set_params(model = KerasRegressor(build_fn=create_model, verbose=0))
                # add the keras model inside the pipeline object
                model_trained.regressor_.named_steps['model'].model = model_keras
            else:
                model_trained.named_steps['model'].model = model_keras
        df_eval.loc[index, 'model_trained'] = model_trained

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more i

In [ ]:
df_eval = df_eval.sort_values(by='rmse_cv')

In [ ]:
print(df_eval)

In [ ]:
df_eval.sort_values(by='rmse_cv').head()

In [ ]:
import plotly.figure_factory as ff

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(
    [y_test, df_eval['model_trained'].iloc[0].predict(x_test)],
    ['Real', 'Predicted'],
    # nbins=50,
    bin_size=150,
    # show_hist=False
    # colors=['#636EFA', '#EF553B']
)
    # show_hist=False)

# Add title
fig.update_layout(title_text='Predicted and Real Value')
fig.show()